In [1]:
### Import Python Package ###
import pandas as pd
import numpy as np
import pytz

In [2]:
### Import Raw Data ###
rides_raw_data = pd.read_csv("Data/rides.csv")
country_code_raw_data = pd.read_csv("Data/country_code.csv")

/var/folders/2t/9mggmpbd4hj3qznywmbrt2_r0000gn/T/ipykernel_10478/2199318435.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  rides_raw_data = pd.read_csv("Data/rides.csv")


In [3]:
### Preprocess Raw Data ####
## 1. Remove all records for rides in Egpy (EG) and QATAR (QA)
rides_data = rides_raw_data[~rides_raw_data['country_code'].isin(['EG','QA'])]

## 2. Transform passenger_id and driver_id to user friendly format
# Create unique mappings for passenger and driver IDs
passenger_ids = rides_data['passenger_id'].unique()
driver_ids = rides_data['driver_id'].unique()

# Create dictionaries for mapping
passenger_map = {uuid: f'passenger {i+1}' for i, uuid in enumerate(passenger_ids)}
driver_map = {uuid: f'driver {i+1}' for i, uuid in enumerate(driver_ids)}

# Map the UUIDs to the new identifiers
rides_data['passenger_id_new'] = rides_data['passenger_id'].map(passenger_map)
rides_data['driver_id_new'] = rides_data['driver_id'].map(driver_map)

## 3. Convert datetime columns to America/Sao_Paulo time
# Define the UTC and Latin America time zones
utc = pytz.UTC
latin_america_tz = pytz.timezone('America/Sao_Paulo')

# Convert the requested_at column to datetime with timezone awareness, handling null values
rides_data['requested_at'] = pd.to_datetime(rides_data['requested_at'], utc=True, errors='coerce')
rides_data['payment_created_at'] = pd.to_datetime(rides_data['payment_created_at'], utc=True, errors='coerce')
rides_data['boarded_at'] = pd.to_datetime(rides_data['boarded_at'], utc=True, errors='coerce')

# Convert the datetime column to the Latin America time zone in a vectorized manner
rides_data['requested_at'] = rides_data['requested_at'].dt.tz_convert(latin_america_tz)
rides_data['payment_created_at'] = rides_data['payment_created_at'].dt.tz_convert(latin_america_tz)
rides_data['boarded_at'] = rides_data['boarded_at'].dt.tz_convert(latin_america_tz)

## 4. Calculate the pickup_duration from boarded_at and requested_at
rides_data['pickup_duration_in_min'] = rides_data.apply(
    lambda row: (row['boarded_at'] - row['requested_at']).total_seconds()/60 if pd.notnull(row['boarded_at']) else None,
    axis=1
)

## 5. Join with country code data to get the country name
rides_final = pd.merge(rides_data,country_code_raw_data, on=['country_code'])

/var/folders/2t/9mggmpbd4hj3qznywmbrt2_r0000gn/T/ipykernel_10478/1877655866.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rides_data['passenger_id_new'] = rides_data['passenger_id'].map(passenger_map)
/var/folders/2t/9mggmpbd4hj3qznywmbrt2_r0000gn/T/ipykernel_10478/1877655866.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rides_data['driver_id_new'] = rides_data['driver_id'].map(driver_map)
/var/folders/2t/9mggmpbd4hj3qznywmbrt2_r0000gn/T/ipykernel_10478/1877655866.py:24: SettingWithCopyWarning

In [4]:
rides_final

,ride_id,city_code,country_code,passenger_id,requested_at,payment_created_at,boarded_at,driver_id,payment_final_value,rating_stars,passenger_id_new,driver_id_new,pickup_duration_in_min,country,country_code_a3,region
0,EC0JAff51nY3MdlULIUdSh1M,NIT,BR,aUdWsZZwy9LpLO9ZpHu4HUtV,2018-05-01 13:00:16-03:00,2018-05-01 13:12:28-03:00,2018-05-01 13:08:03-03:00,7+VqXxCMdlLOvNwk88DU9Vf7,5.97,NaN,passenger 1,driver 1,7.783333,BRAZIL,BRA,LACNIC
1,6Dbb9cEX0IohVU0eDEsBnDXr,CWB,BR,fPX9FKvigi/ooFynRXqz34hY,2018-05-01 01:38:06-03:00,2018-05-01 01:50:14-03:00,2018-05-01 01:42:20-03:00,NpW8rnnQk1IaXf0LOR1ehVUb,14.79,NaN,passenger 42,driver 42,4.233333,BRAZIL,BRA,LACNIC
2,DSzMG9WttV3DMCP+EkkpU6L8,AJU,BR,OMRWtKYb11zFxqy25wWF/WoG,2018-04-30 20:31:54-03:00,2018-04-30 20:43:17-03:00,2018-04-30 20:35:05-03:00,QrhM54fa4qzO0gHC4AmQnY7s,9.52,NaN,passenger 43,driver 43,3.183333,BRAZIL,BRA,LACNIC
3,uFjusCEro441pqaSOh4FnA2b,MCZ,BR,vLrgaHTAz3L5FOlCd35nQxrQ,2018-05-01 11:22:40-03:00,2018-05-01 11:33:26-03:00,2018-05-01 11:26:54-03:00,llGqoxVd8uCKW7hL+Rzj5vQG,9.02,5.0,passenger 44,driver 44,4.233333,BRAZIL,BRA,LACNIC
4,Z9H9pIQRskipiy35UsYs3pMo,SSA,BR,WDrOgbeYlJ4dAJZOteH2WBeZ,2018-05-01 11:30:26-03:00,2018-05-01 11:56:27-03:00,2018-05-01 11:35:18-03:00,a56GQRmr3/MiibgO5UCZ7qeB,18.71,NaN,passenger 45,driver 45,4.866667,BRAZIL,BRA,LACNIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485684,u+bK3efmLWTTQpOXU+uDk+zz,SAN,SV,SbGzCG3qX7s8tAu68imJcJRd,2018-05-04 12:01:54-03:00,NaT,NaT,NaN,NaN,NaN,passenger 273499,driver 72985,NaN,EL SALVADOR,SLV,LACNIC
1485685,PlPocNVyoGnqkiQS7pyYXiNz,SAN,SV,SbGzCG3qX7s8tAu68imJcJRd,2018-05-04 12:05:27-03:00,NaT,NaT,NaN,NaN,NaN,passenger 273499,driver 72985,NaN,EL SALVADOR,SLV,LACNIC
1485686,7LdoEHgfVgXtlUn2Oprg1eut,SAN,SV,SbGzCG3qX7s8tAu68imJcJRd,2018-05-04 12:01:53-03:00,NaT,NaT,NaN,NaN,NaN,passenger 273499,driver 72985,NaN,EL SALVADOR,SLV,LACNIC
1485687,0jKFTx5ZujZ6AFuWs93HGexk,SAN,SV,SbGzCG3qX7s8tAu68imJcJRd,2018-05-04 11:58:13-03:00,NaT,NaT,NaN,NaN,NaN,passenger 273499,driver 72985,NaN,EL SALVADOR,SLV,LACNIC


In [5]:
### Export preprocessed data for data viz ###
# rides_final.to_csv('Data/rides_preprocessed.csv')